In [1]:
%run Imports.ipynb

/Users/yashasvi/opt/anaconda3/envs/itp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conf = pyspark.SparkConf()
spark = SparkSession.builder.appName("Rank").getOrCreate()

23/04/18 19:35:35 WARN Utils: Your hostname, Yashasvis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.232 instead (on interface en0)
23/04/18 19:35:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/18 19:35:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df1=spark.read.parquet("/Users/yashasvi/Documents/CleanDuneData/*")

In [4]:
df1.createOrReplaceTempView("Load")

Main Query

In [5]:
df_all = spark.sql("SELECT * FROM ( SELECT n1.buyer, n1.original_currency, n1.nft_token_id, n1.nft_contract_address, n1.original_amount as Bought, n2.original_amount as Sold, n1.block_time as Bought_Timestamp, n2.block_time as Sold_Timestamp,n1.usd_amount as Bought_amount, n2.usd_amount as Sold_amount, (n2.usd_amount - n1.usd_amount) as net, n1.platform as n1platform, n2.platform as n2platform FROM Load n1, Load n2 where n1.buyer = n2.seller AND  n2.block_time > n1.block_time AND  n1.nft_token_id is not null AND n1.nft_token_id = n2.nft_token_id AND n1.nft_contract_address = n2.nft_contract_address) holdings")

# Ranking 1 - average amount spent (here profit , loss automatically done)

In [6]:
df_avg = df_all.groupBy("buyer").agg(avg("net").alias("avg_spent"))
# score1 based on avg_spent
df_rule_avg_spent_pd=df_avg.toPandas()

# Calculate percentiles for customers with avg_spent > 0
df_positive = df_rule_avg_spent_pd[df_rule_avg_spent_pd['avg_spent'] > 0]
percentiles = df_positive['avg_spent'].rank(pct=True)
df_positive['Score1'] = percentiles * 100

# Set score to 0 for customers with avg_spent < 0
df_negative = df_rule_avg_spent_pd[df_rule_avg_spent_pd['avg_spent'] <= 0]
df_negative['Score1'] = 0

# Combine positive and negative dataframes
df_combined = pd.concat([df_positive, df_negative], axis=0)
df_rule_avg_spent=spark.createDataFrame(df_combined)

23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:10 WARN RowBasedKeyValueBatch: Calling spill() on

# Ranking 2 - number of transactions

In [7]:
df_count = df_all.groupBy("buyer").agg(count("*").alias("number_txns"))

# score2 based on number_txns
df_rule_number_txns_pd=df_count.toPandas()

percentiles = df_rule_number_txns_pd['number_txns'].rank(pct=True)
df_rule_number_txns_pd['Score2'] = percentiles * 100
df_rule_number_txns=spark.createDataFrame(df_rule_number_txns_pd)

23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:36:30 WARN RowBasedKeyValueBatch: Calling spill() on

# Diversity Ranking 3 - Number of Nft contract address held

In [9]:
df_num_nftcontract = df_all.groupBy("buyer").agg(countDistinct("nft_contract_address").alias("num_nftcontract"))
# score3 based on num_nftcontract
df_rule_num_nftcontract_pd=df_num_nftcontract.toPandas()

percentiles = df_rule_num_nftcontract_pd['num_nftcontract'].rank(pct=True)
df_rule_num_nftcontract_pd['Score3'] = percentiles * 100
#converting to sparkdf
df_rule_num_nftcontract=spark.createDataFrame(df_rule_num_nftcontract_pd)

# Diversity Ranking 4 - Number of unique currencies transaction done in

In [11]:
df_currency = df_all.groupBy("buyer").agg(countDistinct("original_currency").alias("num_currency"))
# score4 based on num_currency
df_rule_num_currency_pd=df_currency.toPandas()
percentiles = df_rule_num_currency_pd['num_currency'].rank(pct=True)
df_rule_num_currency_pd['Score4'] = percentiles * 100
#converting back to spark df
df_rule_num_currency=spark.createDataFrame(df_rule_num_currency_pd)

# Diversity Ranking 5 - Average Duration (in seconds)

In [12]:
df_all = df_all.withColumn("duration", (unix_timestamp(col("Sold_Timestamp")) - unix_timestamp(col("Bought_Timestamp"))))

In [13]:
df_avg_duration = df_all.groupBy("buyer").agg(avg("duration").alias("avg_duration"))
# score4 based on num_currency
df_rule_avg_duration_pd=df_avg_duration.toPandas()

percentiles = df_rule_avg_duration_pd['avg_duration'].rank(pct=True)
df_rule_avg_duration_pd['Score5'] = percentiles * 100
#converting to sparkdf
df_rule_avg_duration=spark.createDataFrame(df_rule_avg_duration_pd)

23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/04/18 19:38:19 WARN RowBasedKeyValueBatch: Calling spill() on

# Ranking 6 - Frequency of each transaction

In [15]:
from pyspark.sql.functions import split
df_all = df_all.withColumn("date", split("Bought_Timestamp", " ")[0])
df_all = df_all.withColumn("time", split("Bought_Timestamp", " ")[1])

In [16]:
df_frequency = df_all.groupBy("buyer").agg(countDistinct("time").alias("frequency"))
df_rule_frequency_pd = df_frequency.toPandas()

percentiles = df_rule_frequency_pd['frequency'].rank(pct=True)
df_rule_frequency_pd['Score6'] = percentiles * 100
#converting to sparkdf
df_rule_frequency=spark.createDataFrame(df_rule_frequency_pd)

# Feature Extraction for Model

In [19]:
joined_scores = df_rule_avg_spent.join(df_rule_number_txns, on='buyer', how='inner').join(df_rule_num_nftcontract, on='buyer', how='inner').join(df_rule_num_currency, on='buyer', how='inner').join(df_rule_avg_duration, on='buyer', how='inner').join(df_rule_frequency, on='buyer', how='inner')

In [ ]:
joined_scores.write.parquet("/Users/yashasvi/Documents/ModelScores/")


# Time Series Feature

In [20]:
joined_df = df_rule_avg_spent.join(df_rule_number_txns, on='buyer', how='inner').join(df_rule_num_nftcontract, on='buyer', how='inner').join(df_rule_num_currency, on='buyer', how='inner').join(df_rule_frequency, on='buyer', how='inner').join(df_rule_avg_duration, on='buyer', how='inner')


In [21]:
#filtering records for past 1 year and past 2 years
df_all_filtered_1yearago = df_all.filter(df_all['Bought_Timestamp'] < date_sub(current_date(), 365))
df_all_filtered_2yearsago = df_all.filter(df_all['Bought_Timestamp'] < date_sub(current_date(), 365*2))

In [70]:
df_avg_spent_1 = df_all_filtered_1yearago.groupBy("buyer").agg(avg("net").alias("avg_spent_1"))
df_avg_spent_2 = df_all_filtered_2yearsago.groupBy("buyer").agg(avg("net").alias("avg_spent_2"))

In [71]:
def yearwise(df_avg,colname,scorename):
    df_rule_avg_spent_pd=df_avg.toPandas()
    df_positive = df_rule_avg_spent_pd[df_rule_avg_spent_pd[colname] > 0]
    percentiles = df_positive[colname].rank(pct=True)
    df_positive[scorename] = percentiles * 100
    df_negative = df_rule_avg_spent_pd[df_rule_avg_spent_pd[colname] <= 0]
    df_negative[scorename] = 0
    df_combined = pd.concat([df_positive, df_negative], axis=0)
    df_spark=spark.createDataFrame(df_combined)
    return df_spark

In [73]:
df_rule_avg_1 = yearwise(df_avg_spent_1, 'avg_spent_1','Score1_1')
df_rule_avg_2 = yearwise(df_avg_spent_2, 'avg_spent_2','Score1_2')

In [33]:
df_count_1 = df_all_filtered_1yearago.groupBy("buyer").agg(count("*").alias("number_txns_1"))
df_count_2 = df_all_filtered_2yearsago.groupBy("buyer").agg(count("*").alias("number_txns_2"))

In [34]:
def yearwiseranking2(df_count, colname, scorename):
    df_rule_number_txns_pd=df_count.toPandas()
    percentiles = df_rule_number_txns_pd[colname].rank(pct=True)
    df_rule_number_txns_pd[scorename] = percentiles * 100
    df_rule_number_txns=spark.createDataFrame(df_rule_number_txns_pd)
    return df_rule_number_txns

In [36]:
df_rule_number_txns_1 = yearwiseranking2(df_count_1, 'number_txns_1','Score2_1')
df_rule_number_txns_2 = yearwiseranking2(df_count_2, 'number_txns_2','Score2_2')

In [39]:
df_num_nftcontract_1 = df_all_filtered_1yearago.groupBy("buyer").agg(countDistinct("nft_contract_address").alias("num_nftcontract_1"))
df_num_nftcontract_2 = df_all_filtered_2yearsago.groupBy("buyer").agg(countDistinct("nft_contract_address").alias("num_nftcontract_2"))

In [40]:
def yearwiseranking3(df_num_nftcontract, colname, scorename):
    df_rule_num_nftcontract_pd=df_num_nftcontract.toPandas()
    percentiles = df_rule_num_nftcontract_pd[colname].rank(pct=True)
    df_rule_num_nftcontract_pd[scorename] = percentiles * 100
    df_rule_num_nftcontract=spark.createDataFrame(df_rule_num_nftcontract_pd)
    return df_rule_num_nftcontract

In [42]:
df_rule_num_nftcontract_1 = yearwiseranking3(df_num_nftcontract_1, 'num_nftcontract_1','Score3_1')
df_rule_num_nftcontract_2 = yearwiseranking3(df_num_nftcontract_2, 'num_nftcontract_2','Score3_2')

In [43]:
df_currency_1 = df_all_filtered_1yearago.groupBy("buyer").agg(countDistinct("original_currency").alias("num_currency_1"))
df_currency_2 = df_all_filtered_2yearsago.groupBy("buyer").agg(countDistinct("original_currency").alias("num_currency_2"))

In [44]:
def yearwiseranking4(df_currency, colname, scorename):
    df_rule_num_currency_pd=df_currency.toPandas()
    percentiles = df_rule_num_currency_pd[colname].rank(pct=True)
    df_rule_num_currency_pd[scorename] = percentiles * 100
    df_rule_num_currency=spark.createDataFrame(df_rule_num_currency_pd)
    return df_rule_num_currency

In [46]:
df_rule_num_currency_1 = yearwiseranking4(df_currency_1, 'num_currency_1','Score4_1')
df_rule_num_currency_2 = yearwiseranking4(df_currency_2, 'num_currency_2','Score4_2')

In [48]:
df_avg_duration_1 = df_all_filtered_1yearago.groupBy("buyer").agg(avg("duration").alias("avg_duration_1"))
df_avg_duration_2 = df_all_filtered_2yearsago.groupBy("buyer").agg(avg("duration").alias("avg_duration_2"))

In [49]:
def yearwiseranking5(df_avg_duration, colname, scorename):
    df_rule_avg_duration_pd=df_avg_duration.toPandas()
    percentiles = df_rule_avg_duration_pd[colname].rank(pct=True)
    df_rule_avg_duration_pd[scorename] = percentiles * 100
    df_rule_avg_duration=spark.createDataFrame(df_rule_avg_duration_pd)
    return df_rule_avg_duration

In [52]:
df_rule_avg_duration_1 = yearwiseranking5(df_avg_duration_1, 'avg_duration_1','Score5_1')
df_rule_avg_duration_2 = yearwiseranking5(df_avg_duration_2, 'avg_duration_2','Score5_2')

In [64]:
df_frequency_1 = df_all_filtered_1yearago.groupBy("buyer").agg(countDistinct("time").alias("frequency_1"))
df_frequency_2 = df_all_filtered_2yearsago.groupBy("buyer").agg(countDistinct("time").alias("frequency_2"))

In [67]:
def yearwiseranking6(df_frequency, colname, scorename):
    df_rule_frequency_pd = df_frequency.toPandas()
    percentiles = df_rule_frequency_pd[colname].rank(pct=True)
    df_rule_frequency_pd[scorename] = percentiles * 100
    df_rule_frequency=spark.createDataFrame(df_rule_frequency_pd)
    return df_rule_frequency

In [68]:
df_rule_frequency_1 = yearwiseranking6(df_frequency_1, 'frequency_1','Score6_1')
df_rule_frequency_2 = yearwiseranking6(df_frequency_2, 'frequency_2','Score6_2')

In [78]:
final_datframe = joined_df.join(df_rule_avg_1, on='buyer', how='outer').join(df_rule_avg_2, on='buyer', how='outer').join(df_rule_number_txns_1, on='buyer', how='outer').join(df_rule_number_txns_2, on='buyer', how='outer').join(df_rule_num_nftcontract_1, on='buyer', how='outer').join(df_rule_num_nftcontract_2, on='buyer', how='outer').join(df_rule_num_currency_1, on='buyer', how='outer').join(df_rule_num_currency_2, on='buyer', how='outer').join(df_rule_avg_duration_1, on='buyer', how='outer').join(df_rule_avg_duration_2, on='buyer', how='outer').join(df_rule_frequency_1, on='buyer', how='outer').join(df_rule_frequency_2, on='buyer', how='outer')

In [79]:
final_datframe.write.parquet("/Users/yashasvi/Documents/yearwisedistribution/")

23/04/18 22:11:07 WARN TaskSetManager: Stage 558 contains a task of very large size (1435 KiB). The maximum recommended task size is 1000 KiB.
23/04/18 22:11:07 WARN TaskSetManager: Stage 559 contains a task of very large size (1280 KiB). The maximum recommended task size is 1000 KiB.
23/04/18 22:11:08 WARN TaskSetManager: Stage 560 contains a task of very large size (1280 KiB). The maximum recommended task size is 1000 KiB.
23/04/18 22:11:08 WARN TaskSetManager: Stage 561 contains a task of very large size (1280 KiB). The maximum recommended task size is 1000 KiB.
23/04/18 22:11:08 WARN TaskSetManager: Stage 562 contains a task of very large size (1280 KiB). The maximum recommended task size is 1000 KiB.
23/04/18 22:11:08 WARN TaskSetManager: Stage 563 contains a task of very large size (1435 KiB). The maximum recommended task size is 1000 KiB.
23/04/18 22:11:08 WARN TaskSetManager: Stage 564 contains a task of very large size (1109 KiB). The maximum recommended task size is 1000 KiB.